In [25]:
from dotenv import load_dotenv
import os
import requests

load_dotenv()

API_KEY = os.getenv("API_KEY")
URL = f"https://api.resrobot.se/v2.1/trip?format=json&originId=740000001&destId=740000003&passlist=true&showPassingPoints=true&accessId={API_KEY}"
response = requests.get(URL)
result= response.json()
result.keys() 


dict_keys(['Trip', 'ResultStatus', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId', 'scrB', 'scrF'])

In [26]:
len(result["Trip"])

9

In [27]:
result["Trip"][0].keys()

dict_keys(['Origin', 'Destination', 'ServiceDays', 'LegList', 'calculation', 'TripStatus', 'idx', 'tripId', 'ctxRecon', 'duration', 'rtDuration', 'checksum'])

In [28]:
result["Trip"][0]["LegList"]["Leg"][0]["Origin"]["name"]


'Stockholm Centralstation'

In [29]:
result["Trip"][0]["LegList"]["Leg"][0]["Destination"]["name"]   

'Malmö Centralstation'

In [30]:
def get_trips(origin_id=740000001, destination_id=740098001):
    """origing_id and destination_id can be found from Stop lookup API"""
    url = f"https://api.resrobot.se/v2.1/trip?format=json&originId={origin_id}&destId={destination_id}&passlist=true&showPassingPoints=true&accessId={API_KEY}"

    try:
        response = requests.get(url)
        response.raise_for_status()

        return response.json()
    except requests.exceptions.RequestException as err:
        print(f"Network or HTTP error: {err}")


result = get_trips()
result.keys()

dict_keys(['Trip', 'ResultStatus', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId', 'scrB', 'scrF'])

In [36]:
# checks first trip keys
example_trip = result["Trip"][0]
example_trip.keys()

dict_keys(['Origin', 'Destination', 'ServiceDays', 'LegList', 'calculation', 'TripStatus', 'idx', 'tripId', 'ctxRecon', 'duration', 'rtDuration', 'checksum', 'transferCount'])

In [39]:
example_trip['Origin']

{'name': 'Stockholm Centralstation',
 'type': 'ST',
 'id': 'A=1@O=Stockholm Centralstation@X=18058151@Y=59330136@U=1@L=740000001@',
 'extId': '740000001',
 'lon': 18.058151,
 'lat': 59.330136,
 'routeIdx': 0,
 'prognosisType': 'PROGNOSED',
 'time': '12:41:00',
 'date': '2025-01-07',
 'minimumChangeDuration': 'PT20M'}

In [40]:
example_trip['Destination']

{'name': 'Göteborg Centralstation',
 'type': 'ST',
 'id': 'A=1@O=Göteborg Centralstation@X=11973479@Y=57708895@U=1@L=740000002@',
 'extId': '740000002',
 'lon': 11.973479,
 'lat': 57.708895,
 'routeIdx': 18,
 'prognosisType': 'PROGNOSED',
 'time': '17:00:00',
 'date': '2025-01-07',
 'minimumChangeDuration': 'PT15M'}

In [44]:
example_trip['LegList']['Leg'][0].keys()

dict_keys(['Origin', 'Destination', 'Notes', 'JourneyDetailRef', 'JourneyStatus', 'Product', 'Stops', 'JourneyDetail', 'id', 'idx', 'name', 'number', 'category', 'type', 'reachable', 'waitingState', 'direction', 'directionFlag', 'duration', 'minimumChangeDuration'])

In [48]:
example_stops = example_trip['LegList']['Leg'][0]['Stops']['Stop']
example_stops

[{'name': 'Stockholm Centralstation',
  'id': 'A=1@O=Stockholm Centralstation@X=18058151@Y=59330136@U=1@L=740000001@',
  'extId': '740000001',
  'routeIdx': 0,
  'lon': 18.058151,
  'lat': 59.330136,
  'depPrognosisType': 'PROGNOSED',
  'depTime': '12:41:00',
  'depDate': '2025-01-07',
  'depDir': 'Karlstad Centralstation',
  'minimumChangeDuration': 'PT20M'},
 {'Notes': {'Note': [{'value': 'Stannar endast för påstigande',
     'key': 'text.realtime.stop.exit.disabled',
     'type': 'R',
     'txtN': 'Stannar endast för påstigande'}]},
  'name': 'Södertälje Syd station',
  'id': 'A=1@O=Södertälje Syd station@X=17645367@Y=59162415@U=1@L=740000055@',
  'extId': '740000055',
  'routeIdx': 1,
  'lon': 17.645367,
  'lat': 59.162415,
  'depTime': '13:01:00',
  'depDate': '2025-01-07',
  'alighting': False,
  'minimumChangeDuration': 'PT10M'},
 {'name': 'Katrineholm Centralstation',
  'id': 'A=1@O=Katrineholm Centralstation@X=16208324@Y=58996591@U=1@L=740000166@',
  'extId': '740000166',
  'r

In [54]:
for stop in example_stops:
    print(stop['name'])


Stockholm Centralstation
Södertälje Syd station
Katrineholm Centralstation
Hallsberg station


In [55]:
[{stop.get("name"): stop.get("arrTime")} for stop in example_stops]

[{'Stockholm Centralstation': None},
 {'Södertälje Syd station': None},
 {'Katrineholm Centralstation': '13:43:00'},
 {'Hallsberg station': '14:19:00'}]

# Stop lookup API to find id

In [ ]:
location = "göteborg"
url = f"https://api.resrobot.se/v2.1/location.name?input={location}&format=json&accessId={API_KEY}"
response = requests.get(url)
result = response.json()
result.keys()

dict_keys(['stopLocationOrCoordLocation', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'requestId'])

In [63]:
stop_locations = result["stopLocationOrCoordLocation"]
len(stop_locations)

10

In [61]:
stop_locations[0]

{'StopLocation': {'productAtStop': [{'icon': {'res': 'prod_gen'}, 'cls': '2'},
   {'icon': {'res': 'prod_gen'}, 'cls': '4'},
   {'icon': {'res': 'prod_gen'}, 'cls': '8'},
   {'icon': {'res': 'prod_gen'}, 'cls': '16'},
   {'icon': {'res': 'prod_gen'}, 'cls': '64'},
   {'icon': {'res': 'prod_gen'}, 'cls': '128'},
   {'icon': {'res': 'prod_gen'}, 'cls': '256'}],
  'timezoneOffset': 60,
  'id': 'A=1@O=GÖTEBORG@X=11973479@Y=57708895@U=1@L=740098001@B=1@p=1736224604@',
  'extId': '740098001',
  'name': 'GÖTEBORG',
  'lon': 11.973479,
  'lat': 57.708895,
  'weight': 15424,
  'products': 222,
  'meta': True,
  'minimumChangeDuration': 'PT26M'}}

In [64]:
stop_locations[0]["StopLocation"].keys()

dict_keys(['productAtStop', 'timezoneOffset', 'id', 'extId', 'name', 'lon', 'lat', 'weight', 'products', 'meta', 'minimumChangeDuration'])

In [67]:
stop_locations[0]["StopLocation"]["name"], stop_locations[0]["StopLocation"]["extId"]

('GÖTEBORG', '740098001')